In [79]:
import pandas as pd
from datetime import datetime
import numpy as np
import glob
import os


## Clean up overdose data

In [80]:
orig_cols_to_keep = ["REPORTER_DEA_NO", "REPORTER_STATE", "REPORTER_ZIP", "REPORTER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "CALC_BASE_WT_IN_GM", "DOSAGE_UNIT",
                "TRANSACTION_ID", "Product_Name", "Ingredient_Name", "Measure", "dos_str"]
# keeping more columns than is probably necessary
# just want to make sure we have everything we need, as well as have the opportunity to catch any further filtering we miss at first
pd.set_option('display.max_columns', None)


In [81]:
def find_year(TRANSACTION_DATE):
    """
    Args:
        TRANSACTION_DATE (str): date in format MMDDYYYY

    Returns:
        int: year
    """
    TRANSACTION_DATE = str(TRANSACTION_DATE)
    
    return int(TRANSACTION_DATE[-4:])

# quick test 
find_year(12202019)

2019

In [82]:
# to load in the data, we need to truncate the amount of columns we use as well as the states
cols_to_keep = ["REPORTER_DEA_NO", "REPORTER_STATE", "REPORTER_ZIP", "REPORTER_COUNTY", "DRUG_CODE", "TRANSACTION_CODE", "DRUG_NAME", "QUANTITY", "TRANSACTION_DATE", "Product_Name"]

# we know we need Florida, Texas, and Washington
states = ["FL", "TX", "WA"]
# since we are normalizing based on population, I think we should pick states that are regionally close to one another
# we can change this later as a group, but I have these selected below:
variable_states = ["GA", "OK", "OR"]

# append variable states to our original list
states.extend(variable_states)

# now, load in our data as an iterator so we can load in chunks
it = pd.read_csv("00_source_data/arcos_all_washpost.tsv", chunksize=500_000, sep='\t', usecols = cols_to_keep) # may have to change chunksize depending on your computer's memory

# init empty dataframe
df = pd.DataFrame()

for chunk in it:
    # extract year out of date column
    chunk["year"] = chunk["TRANSACTION_DATE"].apply(lambda x: find_year(x))

    # ensure we're working in the correct date range
    filtered_chunk = chunk[chunk["year"] > 2002]
    filtered_chunk = filtered_chunk[filtered_chunk["year"] < 2016]

    # filter out the states we want
    filtered_chunk = filtered_chunk[filtered_chunk["REPORTER_STATE"].isin(states)]

    df = df.append(filtered_chunk)
    break
df

,REPORTER_DEA_NO,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,TRANSACTION_CODE,DRUG_CODE,DRUG_NAME,QUANTITY,TRANSACTION_DATE,Product_Name,year
6224,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,2.0,11232009,HYDROCODONE BIT/IBUPROFEN 7.5MG/200M,2009
6225,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,2.0,12172012,HYDROCODONE BIT./ACETAMINOPHEN TABS.,2012
6226,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,2192007,HYDROCODONE/IBUPROFEN 7.5MG/200MG TA,2007
6227,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,6102011,HYDROCODONE BIT/ACETA 10MG/325MG USP,2011
6228,PB0167127,GA,31793,TIFT,S,9193,HYDROCODONE,1.0,6022008,HYDROCODONE BIT/ACETA 7.5MG/325MG US,2008
...,...,...,...,...,...,...,...,...,...,...,...
454316,PM0022929,OR,97070,CLACKAMAS,S,9193,HYDROCODONE,2.0,10012012,HYDROCODO.BIT/APAP 7.5MG/750MG USP T,2012
454317,PM0022929,OR,97070,CLACKAMAS,S,9193,HYDROCODONE,1.0,10192012,HYDROCODONE BIT/ACETA 7.5MG/325MG US,2012
454318,PM0022929,OR,97070,CLACKAMAS,S,9143,OXYCODONE,4.0,10252012,OXYCODONE HCL/ACETAMINOPHEN 10MG/325,2012
454319,PM0022929,OR,97070,CLACKAMAS,S,9143,OXYCODONE,3.0,11192012,OXYCODONE HCL/ACETAMINOPHEN 10MG/325,2012


In [83]:
# now that we have our columns and states filtered, let's export this as a csv and store the (much smaller) result on github

df.to_csv("05_cleaned_data/arcos_all_washpost_clean.csv", index=False)

## Clean up cause of death data

In [84]:
path = r'00_source_data/cause_of_death' # use your path
filenames = glob.glob(path + "/*.txt")

df = pd.DataFrame()

for f in filenames:
    temp = pd.read_csv(f, index_col=None, header=0, sep='\t')
    # we're getting some extraneous notes at the bottom - let's just drop for now
    temp.dropna(subset={'County'}, inplace=True)
    
    df = pd.concat([df, temp], axis=0, ignore_index=True)

In [87]:
# helper functions to separate county and state

def abtract_state(county):
    """
    Args:
        county (str): county name

    Returns:
        str: state
    """
    return county.split(", ")[1]



def abstract_county(county):
    """
    Args:
        county (str): county name

    Returns:
        str: county
    """
    return county.split(", ")[0]


df["State"] = df.apply(lambda x: abtract_state(x["County"]), axis=1)
df["County"] = df.apply(lambda x: abstract_county(x["County"]), axis=1)

df.drop(columns={"Notes"}, inplace=True)

In [88]:
df.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State
0,Autauga County,1001.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,397.0,AL
1,Baldwin County,1003.0,2003.0,2003.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,AL
2,Baldwin County,1003.0,2003.0,2003.0,All other alcohol-induced causes,A9,14.0,AL
3,Baldwin County,1003.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,1479.0,AL
4,Barbour County,1005.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,287.0,AL


In [89]:
df.to_csv("05_cleaned_data/cause_of_death_clean.csv", index=False)

## Notes for the group

- haven't filtered out county and state yet as it doesnt seem necessary (small dataset already)